# Task Two: Flow, Density, and Speed Contours

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. Define file path

In [2]:
# Group 2 US101SB
stations_us101_g2_path = "Group 2 US101SB/Stations_US101.csv"
zone_readings_10_19_path = "Group 2 US101SB/Zone_Readings_10-19.csv"
zone_readings_10_26_path = "Group 2 US101SB/Zone_Readings_10-26.csv"

# SR_91_Bottleneck_B_150-200
inventory_sr91_b_path = "SR_91_Bottleneck_B_150-200/Inventory.csv"
zone_readings_0831_0901_path = "SR_91_Bottleneck_B_150-200/Zone_Readings_0831-0901.csv"
zone_readings_0907_0908_path = "SR_91_Bottleneck_B_150-200/Zone_Readings_0907-0908.csv"

## 2. Load data

In [3]:
# Read all zoning readings data

# USB 101 data
zone_readings_10_19 = pd.read_csv(zone_readings_10_19_path)
zone_readings_10_26 = pd.read_csv(zone_readings_10_26_path)

# SR 91 data
zone_readings_0831_0901 = pd.read_csv(zone_readings_0831_0901_path)
zone_readings_0907_0908 = pd.read_csv(zone_readings_0907_0908_path)

## 3. Explore data

In [4]:
# Drop the index
zone_readings_10_19 = zone_readings_10_19.drop(columns=['index'])
zone_readings_10_19.head()

# Rename the sr 91 data
zone_readings_0831_0901 = zone_readings_0831_0901.rename(columns={'measurement_start': 'date', 'zone_id': 'station_ID'})
zone_readings_0907_0908 = zone_readings_0907_0908.rename(columns={'measurement_start': 'date', 'zone_id': 'station_ID'})

In [5]:
# Calculate the density = (52.5 / (l_v * l_D)) * occupancy
lv = 18 # ft
l_D = 6 # ft
zone_readings_10_19['density'] = (52.5 / (lv * l_D)) * zone_readings_10_19['occupancy']
zone_readings_10_26['density'] = (52.5 / (lv * l_D)) * zone_readings_10_26['occupancy']
zone_readings_0831_0901['density'] = (52.5 / (lv * l_D)) * zone_readings_0831_0901['occupancy']
zone_readings_0907_0908['density'] = (52.5 / (lv * l_D)) * zone_readings_0907_0908['occupancy']
zone_readings_10_19.head()

,date,station_ID,volume,speed,occupancy,density
0,2019-10-19 00:00:00,402005,0.0,80.000,0.000000,0.000000
1,2019-10-19 00:00:00,405389,0.0,80.000,0.000000,0.000000
2,2019-10-19 00:00:00,405390,8.0,75.375,0.012800,0.006222
3,2019-10-19 00:00:00,405394,0.0,80.000,0.000000,0.000000
4,2019-10-19 00:00:00,405395,2.0,69.000,0.004467,0.002171


In [6]:
# Select station_ID = '405390'
station = zone_readings_10_19.loc[zone_readings_10_19['station_ID'] == 405390]
station.head()

,date,station_ID,volume,speed,occupancy,density
2,2019-10-19 00:00:00,405390,8.0,75.375000,0.012800,0.006222
147,2019-10-19 00:00:00,405390,3.0,69.666667,0.004750,0.002309
292,2019-10-19 00:01:00,405390,3.0,62.666667,0.005025,0.002443
437,2019-10-19 00:01:00,405390,8.0,68.125000,0.013075,0.006356
582,2019-10-19 00:02:00,405390,4.0,58.500000,0.012650,0.006149


## 4. Define function to create contour

In [7]:
# Genrate countour of given metrics, the metrics are volume, speed, density.
# Sort data by station_ID, date.
# Get metric value of each station_ID.
# Create a new contour_df, row is date, column is station_ID, value is metric value.
def create_contour(df, metric):
    df = df.sort_values(by=['station_ID', 'date']).reset_index(drop=True)
    station_IDs = df['station_ID'].unique()
    for station_ID in station_IDs:
        station = df.loc[df['station_ID'] == station_ID]
        station = station.reset_index(drop=True)
        if station_ID == station_IDs[0]:
            contour_df = pd.DataFrame({'date': station['date'], station_ID: station[metric]})
        else:
            station_metric = pd.DataFrame({station_ID: station[metric]})
            contour_df = pd.concat([contour_df, station_metric], axis=1)

    return contour_df

# Test for function
metrics = ['volume', 'speed', 'density']
contour = create_contour(zone_readings_10_19, metrics[2])
contour.tail()

,date,400000,400011,400143,400304,400382,400392,400460,400513,400525,...,410553,412439,412443,412446,415697,415704,416620,416911,418244,421965
2868,2019-10-19 23:58:00,0.009054,0.009178,0.003986,0.007425,0.012359,0.004995,0.012420,0.008385,0.006951,...,0.003792,0.009868,0.008434,0.004148,0.0,0.0,0.009366,0.003038,0.009463,0.010176
2869,2019-10-19 23:58:00,0.008519,0.007292,0.009722,0.011557,0.009127,0.006283,0.010196,0.009868,0.010325,...,0.005687,0.005687,0.005602,0.007826,0.0,0.0,0.006757,0.004521,0.003079,0.001718
2870,2019-10-19 23:59:00,0.006964,0.011657,0.008050,0.009042,0.014729,0.011545,0.005542,0.009795,0.006281,...,0.008377,0.002309,0.004740,0.005671,0.0,0.0,0.014324,0.003306,0.018197,0.007389
2871,2019-10-19 23:59:00,0.004460,0.008050,0.003733,0.007425,0.008446,0.006210,0.012141,0.007644,0.007671,...,0.003338,0.007851,0.004934,0.008993,0.0,0.0,0.013692,0.000608,0.009641,0.008556
2872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 5. Process the data

In [8]:
metrics = ['volume', 'speed', 'density']

# Create contour file for US101SB 10-19
for metric in metrics:
    contour_df = create_contour(zone_readings_10_19, metric)
    contour_csv_path = f"Group 2 US101SB/{metric}_contour_10-19.csv"
    contour_df.to_csv(contour_csv_path, index=False)

# Create contour file for US101SB 10-26
for metric in metrics:
    contour_df = create_contour(zone_readings_10_26, metric)
    contour_csv_path = f"Group 2 US101SB/{metric}_contour_10-26.csv"
    contour_df.to_csv(contour_csv_path, index=False)

# Create contour file for SR91 0831-0901
for metric in metrics:
    contour_df = create_contour(zone_readings_0831_0901, metric)
    contour_csv_path = f"SR_91_Bottleneck_B_150-200/{metric}_contour_0831-0901.csv"
    contour_df.to_csv(contour_csv_path, index=False)

# Create contour file for SR91 0907-0908
for metric in metrics:
    contour_df = create_contour(zone_readings_0907_0908, metric)
    contour_csv_path = f"SR_91_Bottleneck_B_150-200/{metric}_contour_0907-0908.csv"
    contour_df.to_csv(contour_csv_path, index=False)
